In [0]:
%pip install mlflow
%pip install dlt
%pip install databricks-automl-runtime
%pip install holidays
%pip install xgboost==1.5.0
%pip install sklearn
%pip install numpy
%pip install cloudpickle
%pip install autocorrect
%pip install better_profanity
%pip install geopy
%pip install category-encoders

#Sentiment Analytics On Delta Live Tables using ML
1. **Application                :** Social Media Analytics <br/>
2. **Usecase               :** Performing SQL analytics and Machine Learning algorithms on Twitter data coming incrementally from data lake.<br/>
3. **Notebook Summary      :** This notebook is a part of social media analytics application which creates `Delta Live Tables`.<br/>
4. **Notebook Description  :** Creates `Bronze, Silver and Gold` Delta Live tables to manage raw data, filter raw data and curated data respectively. This notebook also performs `ML operation` on Silver data to analyze sentiment analytics on Twitter Messages.

In [0]:
import dlt
import mlflow
from pyspark.sql.functions import struct
from pyspark.sql.functions import col
from pyspark.sql.types import DateType
from pyspark.sql.types import *
import pyspark.sql.functions as f
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
import re
import itertools
from autocorrect import Speller
spell = Speller(lang='en')
from better_profanity import profanity
import numpy as np
from delta.tables import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType, TimestampType, DecimalType

### Data Clean Up

The most common problem data engineers face is having to address unclean data. Tweets are notoriously hard to parse, but we've done our best and developed a user-defined function (UDF) that takes in a Tweet and performs some cleanup to ensure our Tweets conform to a common format for analysis. By making this part of the Delta Live Tables pipeline, we ensure this UDF can be run in a distributed manner on all of our data, accelerating our ETL process at scale considerably.

In [0]:
# defining regex
const_regex_hyperlink = r'https?:\/\/.\S+'
const_regex_retweet = r'^RT[\s]+'
const_regex_twitter_handle = r'@[\w]*'
const_regex_word_sperator = "([A-Z][a-z]+[^A-Z]*)"
const_hashtag = r'#'

In [0]:
#dictionary consisting of the contraction and the actual value
Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
           "'d":" would","'ve":" have","'re":" are"}


def clean_twitter_text(x):
    # ignore non ascii
    tweet = x.encode('ascii', 'ignore').decode('ascii')
    # remove hyperlinks
    tweet = re.sub(const_regex_hyperlink, "", tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(const_hashtag, '', tweet) 
    # remove old style retweet text "RT"
    tweet = re.sub(const_regex_retweet, '', tweet)    
    # remove twitter handles (@user)
    tweet = re.sub(const_regex_twitter_handle, '', tweet)

    tweet = re.sub(":", '', tweet)
    tweet = tweet.strip()
    
    #separate the words
    tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s])
    
    #One letter in a word should not be present more than twice in continuation
    tweet = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",tweet) if s])
    
    #replace the contractions
    for key,value in Apos_dict.items():
        if key in tweet:
            tweet=tweet.replace(key,value)
    
    return tweet

#creating UDF 
clean_twitter_text_udf = udf(clean_twitter_text)

### Incorporating our Sentiment Analysis ML Model

One goal we want to achieve from our pipeline is applying our newly created sentiment analysis ML model to the Tweets we're processing. With DLT this is as simple as declaring a UDF pointing to our model stored in our MLFlow registry. We can then leverage that UDF in our pipeline, passing in each of our Tweets as input and getting a sentiment score as an output. And extending this to streaming scenarios for real-time inference can be done just a couple of lines of code.

In [0]:
const_model_run_id = spark.sql("select * from mlview").collect()[0].value
model_name = "model"
model_uri = "runs:/{run_id}/{model_name}".format(run_id=const_model_run_id, model_name=model_name)
loaded_model = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri, result_type='string')

###Twitter input data schema

### Delta Live Table Setup

As a data engineer, we can use DLT pipelines to curate our raw Twitter data into useful data assets through filtering, augmentation, and other data processing techniques.

You can define a DLT pipeline in either Python or SQL. A DLT pipeline is declarative - you name and describe the tables you want to create, and then define them using familiar DataFrame syntax or SQL object DML.

In this pipeline, we'll move our Twitter data from our bronze to silver layer, filtering for specific hashtags we're interested in analyzing and applying our "clean up" function; and then enhancing it further by applying our sentiment ML model and creating some aggregated tables of hashtag counts by location for convenient consumption in tools like Power BI.

Once we've defined our pipeline in a notebook, we can configure it to run on a scheduled basis, continuously, or when triggered by the arrival of new data in our lakehouse.

In [0]:
#Twitter Schema

twitterSchema = StructType([    
    StructField("time",TimestampType(),True),
    StructField("hashtag",StringType(),True),
    StructField("tweet",StringType(),True),
    StructField("city",StringType(),True),    
    StructField("username",StringType(),True),
    StructField("retweetcount",IntegerType(),True),
    StructField("favouritecount",IntegerType(),True),
    StructField("sentiment",StringType(),True),
    StructField("sentimentscore",DecimalType(),True),
    StructField("isretweet",IntegerType(),True),
    StructField("hourofday",StringType(),True),
    StructField("language",StringType(),True) 
    ])

In [0]:
const_staging_path = "/mnt/data-source/TwitterDataJsonSource/"
# Bronze Table Setup
@dlt.table(
    comment="Raw data",
    table_properties={
    "quality": "bronze"
    }    
)
@dlt.expect_or_drop("valid_city", "City IS NOT NULL")
def bronze_twitter_historical_data():
        return (
    spark.readStream.format("cloudFiles")
      .option("cloudFiles.format", "json")
      .schema(twitterSchema)
      .load(const_staging_path)
  )


# Silver Table Setup
@dlt.create_table(
  comment="Preparing",  
  table_properties={
    "quality": "silver"
  }    
)
def silver_twitter_historical_data():
  return dlt.readStream("bronze_twitter_historical_data").filter(col("tweet").contains("#fashion") | col("tweet").contains("#beach")| col("tweet").contains("#entertainment")| col("tweet").contains("#gogreen")| col("tweet").contains("#sustainablefashion")| col("tweet").contains("#futuretech")).withColumn("tweet", clean_twitter_text_udf('tweet')).withColumn("Date",to_date("time"))



# Gold Table Setup
@dlt.create_table(
  comment="Predicting", 
  partition_cols = ["Date"],
  table_properties={
    "quality": "gold"
  }
)
def gold_twitter_historical_data():
  return dlt.read("silver_twitter_historical_data").withColumn('MLSentiment', loaded_model('tweet'))



# Gold (Hashtag) Table Setup
@dlt.create_table(
  comment="Aggregating",  
  table_properties={
    "quality": "gold"
  }
)
def gold_twitter_historical_hashtag_data():
  return dlt.read("silver_twitter_historical_data").groupBy("hashtag").count()



# Gold (Retweet) Table Setup
@dlt.create_table(
  comment="Aggregating",  
  table_properties={
    "quality": "gold"
  }
)
def gold_twitter_historical_retweetcount_data():
  return dlt.read("silver_twitter_historical_data").selectExpr("sum(cast(retweetcount as int)) RetweetCount")



# Gold (City vs Hashtag) Table Setup
@dlt.create_table(
  comment="Aggregating",  
  table_properties={
    "quality": "gold"
  }
)
def gold_twitter_historical_city_hashtagcount_data():
  return dlt.read("silver_twitter_historical_data").groupBy("city", "hashtag").count()



#Campaign Analytics

1. **Usecase               :** Performing Campaign analytics on static campaign data coming from snowflake in azure container.<br/>
2. **Notebook Summary      :** This notebook is a part of campaign analytics application which perform `campaign analytics using various pyspark capability`.
3. **Notebook Description  :** Performing Campaign Analytics on Azure Container Files.


###Feature List
1. Data Profiling
2. Total Revenue, Total Revenue Target & Profit 
3. Campaign Run by Per Week 
4. Total Profit by Country Per Week
5. Top Loss-Making Campaign 

The bronze data received for processing is already curated. So, we will derive gold tables from bronze tables.

### Import Libraries

In [0]:
from pyspark.sql.functions import sum as _sum
from pyspark.sql.functions import mean as _mean
from pyspark.sql.functions import max as _max
from pyspark.sql.functions import min as _min
import pyspark.sql.functions as func
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType

###Define the Schema for the input file

In [0]:
campaignSchema = StructType([    
    StructField("Region",StringType(),True),
    StructField("Country",StringType(),True),
    StructField("ProductCategory",StringType(),True),
    StructField("Campaign_ID",IntegerType(),True),    
    StructField("Campaign_Name",StringType(),True),
    StructField("Qualification",StringType(),True),
    StructField("Qualification_Number",StringType(),True),
    StructField("Response_Status",StringType(),True),
    StructField("Responses",FloatType(),True),
    StructField("Cost",FloatType(),True),
    StructField("Revenue",FloatType(),True),
    StructField("ROI",FloatType(),True),
    StructField("Lead_Generation",StringType(),True),
    StructField("Revenue_Target",FloatType(),True),
    StructField("Campaign_Tactic",StringType(),True),
    StructField("Customer_Segment",StringType(),True),
    StructField("Status",StringType(),True),
    StructField("Profit",FloatType(),True),
    StructField("Marketing_Cost",FloatType(),True),
    StructField("CampaignID",IntegerType(),True),
    StructField("CampDate",DateType(),True), 
    StructField("SORTED_ID",IntegerType(),True)])
    

### Load the Campaign Dataset from Data Lake

In [0]:
# Bronze Table Setup
@dlt.table(comment="Raw data")
def bronze_campaign_data():
#   return (spark.table("campaign.campaign_source"))
  return (spark.read.format("csv").option("header",True).schema(campaignSchema).load("/mnt/data-source/Campaign Data/campaign-data2.csv"))

### Total Revenue, Total Revenue Target & Profit

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_country_wise_revenue():
    
    df = dlt.read("bronze_campaign_data").groupBy("Country","Region").agg(_sum("Revenue").alias("Total_Revenue"), _sum("Revenue_Target").alias("Total_Revenue_Target"),_sum("Profit").alias("Total_Profit"),_max("Cost").alias("Max_Cost"),_min("Cost").alias("Min_Cost"))
    df = df.withColumn("Total_Revenue", func.round(df["Total_Revenue"],2)).withColumn("Total_Revenue_Target", func.round(df["Total_Revenue_Target"], 2)).withColumn("Total_Profit", func.round(df["Total_Profit"], 2))
    return df
    

### Campaign Run by Per Week

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_campaign_per_week():
    return dlt.read("bronze_campaign_data") \
    .groupBy(
      "Campaign_Name",
      window("CampDate", "1 week")) \
    .count().orderBy(col("count").desc())

### Total Profit by Country Per Week

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_Total_Profit_by_Country_Per_Week():
    return dlt.read("bronze_campaign_data").select("Region","Country", "Cost", "Profit","CampDate") \
                     .groupBy(window(col("CampDate"), "7 days"), col("Country")) \
                     .agg(sum("Profit").alias('Total_Profit'),) \
                            .orderBy(col("window.start"))

### Top Loss-Making Campaign

In [0]:
# Gold Table Setup
@dlt.table(comment="Raw data")
def gold_Top_Loss_Making_Campaign():
    loss = dlt.read("bronze_campaign_data").select("Campaign_Name","Profit").filter(F.col("Profit") < 0)
    loss = loss.withColumn("Loss_Count", F.when((F.col('Profit') < 0 ) , F.lit(1)).otherwise(F.lit(0)))
#     loss = loss.groupBy('Campaign_Name').sum('Loss_Count')
    return loss

#Campaign Powered by Twitter

In [0]:
   
@dlt.table(
    comment="Campaign powered by Twitter",
    table_properties={
    "quality": "gold"
    }    
) 

def Sentiment_Campaign_Analytics():
    return (
        spark.sql("""
         select a.* from live.gold_twitter_historical_city_hashtagcount_data a,
              live.gold_campaign_per_week b
    where replace(a.hashtag, '#', '') = b.campaign_name
        """)
    )
    

       